In [6]:
import pandas as pd
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq

In [7]:
#TGAC
file_gff = "/home/juan/Desktop/juan/bio/data/TGAC/Triticum_aestivum.TGACv1.39.clean.gff3"
dir_fasta = "/home/juan/Desktop/juan/bio/data/TGAC/by_scaff/"
file_out_fasta = "/home/juan/Desktop/juan/bio/mites/data/tgac_genes.fasta"
padding = 1000

In [8]:
#IWGSC
file_gff = "/home/juan/Desktop/juan/bio/data/IWGSC/42/Triticum_aestivum.IWGSC.42.gff3"
dir_fasta = "/home/juan/Desktop/juan/bio/data/IWGSC/42/by_chr/"
file_out_fasta = "/home/juan/Desktop/juan/bio/mites/data/iwgsc_genes.fasta"
padding = 1000

In [9]:
df_gff = pd.read_csv(file_gff, index_col=False, sep='\t', comment='#')
df_gff.columns = ['seqname', 'source', 'feature', 'start', 'end', 'score', 'strand', 'frame', 'attribute']
print(len(df_gff.index))
df_gff = df_gff[(df_gff.feature == 'gene') | ((df_gff.feature == 'ncRNA_gene'))]
print(len(df_gff.index))

1957743
120744


In [ ]:
records = {}

buffer_seqs = []
for k,v in df_gff.iterrows():
    del_1 = "gene:"
    del_2 = ";"
    gene = v.attribute.split(del_1)[1].split(del_2)[0]
    end = max(v.end, v.start)
    start = min(v.end, v.start)
    start = max(start - padding, 0)
    end += padding
    if not v.seqname in records:
        records[v.seqname] = SeqIO.read(dir_fasta + v.seqname + '.fasta', 'fasta')
    record = records[v.seqname]
    clean_seq = ''.join(str(record.seq).splitlines())
    new_seq = clean_seq[start:end]
    desc = "seq:" + str(v.seqname)
    desc += " start:" + str(start)
    desc += " end:" + str(end)
    desc += " strand:" + str(v.strand)
    seq = SeqRecord(Seq(new_seq), id=gene, description=desc)
    buffer_seqs.append(seq)


In [ ]:
SeqIO.write(buffer_seqs, file_out_fasta, "fasta")
print("%i written to %s" % (len(buffer_seqs), file_out_fasta) )